# 导入tushare

In [ ]:
import tushare as ts

# 初始化pro接口

In [ ]:
pro = ts.pro_api('ec30ba0eb5f07c64bf98162161fd2009bbd3ca48b7c02f3a81b99f97')

# 拉取数据

In [ ]:
todayData = pro.daily(**{
    "ts_code": "",
    "trade_date": "",
    "start_date": "",
    "end_date": "",
    "offset": "",
    "limit": ""
}, fields=[
    "ts_code",
    "trade_date",
    "open",
    "high",
    "low",
    "close",
    "pre_close",
    "change",
    "pct_chg",
    "vol",
    "amount"
])
print(todayData)

In [ ]:
for row in todayData.itertuples():
    print(getattr(row, 'ts_code'))